In [1]:
%pip install opencv-python mediapipe

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import mediapipe as mp
import mediapipe.tasks.python.components.containers
import numpy as np
import pandas as pd
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.stattools import adfuller
from itertools import product
import math

In [4]:
def calculate_length(p1 :tuple[int,int] , p2:tuple[int,int]) -> float:
    """
    두 점 사이의 거리를 계산합니다. (유클라디안 거리)
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :return: 두 점 사이의 거리
    """
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def calculate_angle(p1:tuple[int,int], p2:tuple[int,int], p3:tuple[int,int]):
    """
    삼각형의 세 점 사이의 각도를 계산합니다.
    :param p1: 첫 번째 점 (x, y)
    :param p2: 두 번째 점 (x, y)
    :param p3: 세 번째 점 (x, y)
    :return: 삼각형의 세 점 사이의 각도 ∠p1p2p3
    """
    # 삼각형의 세 점 사이의 각도를 계산합니다.
    a = calculate_length(p1, p2)
    b = calculate_length(p2, p3)
    c = calculate_length(p3, p1)
    # 코사인 법칙을 사용하여 각도를 계산합니다.
    # ∠abc = arccos((a² + b² - c²) / 2ab)
    return math.degrees(math.acos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b)))

In [6]:

def extractSkeleton(video_path, output_path):
    # 1. 미디어파이프를 사용하여 비디오에서 스켈레톤 구조 추출하기
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    # video_path = './videos/Standard.MOV' # 스쿼트
    # video_path = './videos/pushup.mp4' # 푸시업
    cap = cv2.VideoCapture(video_path)  # 비디오 파일 경로
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    pose_data = []
    # cnt = 0
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            height, width, _ = image.shape
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                # 왼쪽 팔 각도 계산
                left_elbow_angle = calculate_angle( (landmarks[11].x * width, landmarks[11].y * height), (landmarks[13].x * width, landmarks[13].y * height), (landmarks[15].x * width, landmarks[15].y * height))
                # 오른쪽 팔 각도 계산
                right_elbow_angle = calculate_angle( (landmarks[12].x * width, landmarks[12].y * height), (landmarks[14].x * width, landmarks[14].y * height), (landmarks[16].x * width, landmarks[16].y * height))
                # 왼쪽 손목 각도 계산
                left_wrist_angle = calculate_angle( (landmarks[13].x * width, landmarks[13].y * height), (landmarks[15].x * width, landmarks[15].y * height), (landmarks[17].x * width, landmarks[17].y * height))
                # 오른쪽 손목 각도 계산
                right_wrist_angle = calculate_angle( (landmarks[14].x * width, landmarks[14].y * height), (landmarks[16].x * width, landmarks[16].y * height), (landmarks[18].x * width, landmarks[18].y * height))
                # 왼쪽 어깨 각도 계산
                left_shoulder_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[11].x * width, landmarks[11].y * height), (landmarks[13].x * width, landmarks[13].y * height))
                # 오른쪽 어깨 각도 계산
                right_shoulder_angle = calculate_angle( (landmarks[24].x * width, landmarks[24].y * height), (landmarks[12].x * width, landmarks[12].y * height), (landmarks[14].x * width, landmarks[14].y * height))
                # 어깨 중앙값 계산
                shoulder_center = (landmarks[11].x * width + landmarks[12].x * width) / 2, (landmarks[11].y * height + landmarks[12].y * height) / 2
                # 엉덩이 중앙값 계산
                hip_center = (landmarks[23].x * width + landmarks[24].x * width) / 2, (landmarks[23].y * height + landmarks[24].y * height) / 2
                # 무릎 중앙값 계산
                knee_center = (landmarks[25].x * width + landmarks[26].x * width) / 2, (landmarks[25].y * height + landmarks[26].y * height) / 2
                # 엉덩이 각도 계산
                hip_angle = calculate_angle(shoulder_center, hip_center, knee_center)
                # 허리 각도 계산
                waist_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[11].x * width, landmarks[11].y * height), (landmarks[24].x * width, landmarks[24].y * height))
                # 왼쪽 무릎 각도 계산
                left_knee_angle = calculate_angle( (landmarks[23].x * width, landmarks[23].y * height), (landmarks[25].x * width, landmarks[25].y * height), (landmarks[27].x * width, landmarks[27].y * height))
                # 오른쪽 무릎 각도 계산
                right_knee_angle = calculate_angle( (landmarks[24].x * width, landmarks[24].y * height), (landmarks[26].x * width, landmarks[26].y * height), (landmarks[28].x * width, landmarks[28].y * height))
                # 왼쪽 발목 각도 계산
                left_ankle_angle = calculate_angle( (landmarks[25].x * width, landmarks[25].y * height), (landmarks[27].x * width, landmarks[27].y * height), (landmarks[29].x * width, landmarks[29].y * height))
                # 오른쪽 발목 각도 계산
                right_ankle_angle = calculate_angle( (landmarks[26].x * width, landmarks[26].y * height), (landmarks[28].x * width, landmarks[28].y * height), (landmarks[30].x * width, landmarks[30].y * height))

                # 각도를 리스트에 저장
                pose_data.append([left_elbow_angle, right_elbow_angle, left_wrist_angle, right_wrist_angle, left_shoulder_angle, right_shoulder_angle, hip_angle, waist_angle, left_knee_angle, right_knee_angle, left_ankle_angle, right_ankle_angle])
    cap.release()
    # 2. 시계열 데이터로 변환
    pose_df = pd.DataFrame(pose_data)
    pose_df.columns = ['left_elbow_angle', 'right_elbow_angle', 'left_wrist_angle', 'right_wrist_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'hip_angle', 'waist_angle', 'left_knee_angle', 'right_knee_angle', 'left_ankle_angle', 'right_ankle_angle']
    pose_df.to_csv(output_path, index=False)
    return pose_df, fps

In [14]:
from scipy.signal import periodogram
# 주기성을 판단하는 함수
def find_dominant_period(data):
    # periodogram: 주파수와 주파수에 해당하는 파워를 계산합니다.(파워: 특정 주파수의 세기)
    freqs, power = periodogram(data, fs = 0.5)
    dominant_freq = freqs[np.argmax(power)]
    dominant_period = 1 / dominant_freq
    return dominant_period, power
def periodicityAnalysis(pose_df,name):
    # pose_df = pd.read_csv("./data/pose3.csv")
    landmark_columns = pose_df.columns
    df_col = [f'{c}_period' for c in landmark_columns] + [
        f'{c}_power' for c in landmark_columns
    ]
    # vel_columns = vel_df.columns
    power_df = pd.DataFrame(columns=df_col)

    for column_name in landmark_columns:
        data = pose_df[column_name].values
        period, power = find_dominant_period(data)
        power_df[f'{column_name}_period'] = [period]
        power_df[f'{column_name}_power'] = [power.max()]
        # new_row = pd.DataFrame([[period, power]], columns=[f'{column_name}_period', f'{column_name}_power'])
        # power_df = pd.concat([power_df, new_row], ignore_index=True)
    pose_df['name'] = name
    return power_df



In [18]:
# subVideos = [f'./subVideos/Lunge{i}.avi' for i in range(27)]
subVideos = [f'./subVideos/Pullup{i}.avi' for i in range(41)]
name = 'Pullup'
out_df = pd.DataFrame()
for video_path in subVideos:
    output_path = f'./data/mediapipe/{video_path[13:-4]}.csv'
    pose_df, fps = extractSkeleton(video_path, output_path)
    power_df = periodicityAnalysis(pose_df,name)
    out_df = pd.concat([out_df, power_df], ignore_index=True)
out_df.to_csv(f'./data/{name}.csv', index=False)


In [19]:
out_df.head()

,left_elbow_angle_period,right_elbow_angle_period,left_wrist_angle_period,right_wrist_angle_period,left_shoulder_angle_period,right_shoulder_angle_period,hip_angle_period,waist_angle_period,left_knee_angle_period,right_knee_angle_period,...,left_wrist_angle_power,right_wrist_angle_power,left_shoulder_angle_power,right_shoulder_angle_power,hip_angle_power,waist_angle_power,left_knee_angle_power,right_knee_angle_power,left_ankle_angle_power,right_ankle_angle_power
0,180.0,180.0,180.0,180.000000,180.0,180.0,90.0,180.0,180.0,180.0,...,4374.287161,3998.103266,132308.779162,136334.295433,299.025605,101.217653,17649.163613,18061.417835,48804.335805,16127.644323
1,180.0,180.0,180.0,90.000000,180.0,180.0,180.0,90.0,60.0,180.0,...,2486.843143,986.283632,124864.927862,116379.052234,344.963307,86.792308,24299.528724,39194.777323,30132.089579,58793.906034
2,180.0,180.0,180.0,180.000000,180.0,180.0,180.0,90.0,90.0,180.0,...,3739.007760,1361.193910,112592.867252,96666.683444,258.888429,39.812251,51442.881848,62957.324116,40615.237469,10329.590796
3,180.0,180.0,180.0,16.363636,180.0,180.0,22.5,60.0,180.0,180.0,...,2053.928397,207.647635,80156.613027,59943.376297,603.102225,80.465258,66967.026575,219728.686837,57744.087732,65223.692076
4,90.0,90.0,60.0,180.000000,90.0,90.0,90.0,180.0,180.0,60.0,...,1820.682213,8861.193362,85903.021884,332978.787927,1301.755075,151.212589,8731.623491,4920.951514,6973.302045,6276.058356
